In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [ ]:
vectorizer = CountVectorizer(stop_words=stopwords.words('spanish')) 

In [ ]:
analyze = vectorizer.build_analyzer()

In [4]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)



In [8]:
from nltk.corpus import wordnet as wn
import spacy
nlp = spacy.load('es_core_news_sm')
def lemmatizer(text):  
    doc = nlp(text)
    return [word.lemma_ for word in doc]
def funcionCoincidencia2(oracion1, oracion2):
    lista1 = []
    lista2 = []
    sinLasStop1 = analyze(oracion1)
    sinLasStop2 = analyze(oracion2)
    #sinLasStop1 = ' '.join(analyze(oracion1))
    #sinLasStop2 = ' '.join(analyze(oracion2))
    #sinLasStop1 = ' '.join(lemmatizer(sinLasStop1))
    #sinLasStop2 = ' '.join(lemmatizer(sinLasStop2))
    #sinLasStop1 = lemmatizer(sinLasStop1) #lemmatizo dos veces para mejores resultados
    #sinLasStop2 = lemmatizer(sinLasStop2)
    for element in sinLasStop1:
        syn1 = wn.synsets(element,lang="spa")
        for element in syn1:
            element.lemmas(lang="spa") #busco sinonimos
            lista1.append([str(lemma.name()) for lemma in element.lemmas(lang="spa")]) #los pongo en una lista
    lista1.append(sinLasStop1) #por si no encuentro sinonimos
    flat_set1 = set([item for sublist in lista1 for item in sublist]) #elimino repetidos
    for element in sinLasStop2: #repetir proceso con la oracion 2
        syn2 = wn.synsets(element,lang="spa")
        for element2 in syn2:
            element2.lemmas(lang="spa")
            lista2.append([str(lemma.name()) for lemma in element2.lemmas(lang="spa")])
    lista2.append(sinLasStop2) #Por si no encuentro sinonimos
    flat_set2 = set([item for sublist in lista2 for item in sublist])
    
    oracion1SinLasStop = " ".join(flat_set1) #las convierto en una oracion
    oracion2SinLasStop = " ".join(flat_set2)

    
    vector1= text_to_vector(oracion1SinLasStop)
    vector2= text_to_vector(oracion2SinLasStop)
    return get_cosine(vector1, vector2) #funcion coseno


In [6]:
import glob
#glob.glob(r"C:\Users\Bernkastel\Downloads\dataset-nlp-plagio-utn-20200715T062020Z-001\dataset-nlp-plagio-utn\*")
listaDocx= glob.glob(r"dataset-nlp-plagio-utn\*")
listaDocx

['dataset-nlp-plagio-utn\\AnyConv.com__Lopez_Tomas_-_TP_6_-_Sistemas_Emergentes.docx',
 'dataset-nlp-plagio-utn\\AnyConv.com__preguntas_TP_Wikinomics_-_Gariglio.docx',
 'dataset-nlp-plagio-utn\\AnyConv.com__TP04_-_TP_Prince_Difusión_y_adopción_TIC.docx',
 'dataset-nlp-plagio-utn\\AnyConv.com__TP06_-__Johnson_Sistemas_emergentes_-_preguntas.docx',
 'dataset-nlp-plagio-utn\\AnyConv.com__Tp1-MKT-Romano_Karra.docx',
 'dataset-nlp-plagio-utn\\AnyConv.com__TP1_-_La_Larga_Cola_-_Chris_Anderson.docx',
 'dataset-nlp-plagio-utn\\AnyConv.com__TP1_-_Wikinomics_-_Santiago_Peralta.docx',
 'dataset-nlp-plagio-utn\\AnyConv.com__TP2_-_La_Larga_Cola_de_Chris_Anderson_corto_-_Santiago_Peralta_.docx',
 'dataset-nlp-plagio-utn\\AnyConv.com__TP2_MKT_-_Gariglio.docx',
 'dataset-nlp-plagio-utn\\AnyConv.com__TP3_Economia_de_Experiencia_-_Vit_Sabrina.docx',
 'dataset-nlp-plagio-utn\\AnyConv.com__TP4_-_Comercio_electronico.docx',
 'dataset-nlp-plagio-utn\\AnyConv.com__TP4_-_Difusión_y_adopción_-_Parte_I_y_II_-_S

In [7]:
import textract
from nltk import tokenize
def textosEnOraciones():
    encoding = "utf-8"
    listaDeTextosEnOraciones = []
    for element in listaDocx:
        text = textract.process(element)
        textoPiola = text.decode(encoding).replace("\n",". ") 
        textoEnOraciones = tokenize.sent_tokenize(textoPiola)
        textoEnOraciones = [item for item in textoEnOraciones if len(item)>=30] #solo frases largas
        textoEnOraciones = [item for item in textoEnOraciones if (item[0:4]!="http")] #le saco los links
        listaDeTextosEnOraciones.append(textoEnOraciones)
    return listaDeTextosEnOraciones
    #print(textoEnOraciones)
#listaDeTextosEnOraciones[0]


In [32]:
from nltk.tag import StanfordPOSTagger
jar = r'C:\Users\Bernkastel\Downloads\stanford-tagger-4.0.0\stanford-tagger-4.0.0\stanford-postagger-4.0.0.jar'
model = r'C:\Users\Bernkastel\Downloads\stanford-tagger-4.0.0\stanford-tagger-4.0.0\models\spanish-ud.tagger'
import os
java_path = r"C:\Program Files\Java\jdk-14\bin\java.exe"
os.environ['JAVAHOME'] = java_path
def verFrasesPlagiadas(path,confianza):
    flagPlagio=0
    encoding = "utf-8"
    text = textract.process(path)
    textoPiola = text.decode(encoding).replace("\n",". ")
    textoPiola = textoPiola.replace("\u200b“"," ")
    textoParaBuscarNombre = textoPiola.replace("."," ")
    textoParaBuscarNombre = textoParaBuscarNombre.replace(","," ")
    textoParaBuscarNombre = textoParaBuscarNombre.replace(":"," ")
    pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8' )
    res = pos_tagger.tag(textoParaBuscarNombre.split())
    nombre = ""
    flag = 0
    for (a,b) in res:
        if(b=='PROPN' and len(a)>4 and a.lower()!='economia' and a.lower()!='marketing' and a.lower()!= 'internet' and a.lower()!= 'economía' and a.lower()!='nueva' and a.lower()!='utn' 
          and a.lower()!='frba' and a.lower()!='tp' and a.lower()!="ing" and a.lower()!="practico" and a.lower()!="emergentes" and a.lower()!="integrantes" and a.lower()!= "legajo" and a.lower()!="legajo" 
          and a.lower()!='sistemas' and a.lower()!='informacion' and a.lower()!='información' and a.lower()!="nombre" and a.lower()!="e-mail" and a.lower()!= "ingeniería" and a.lower()!="ingenieria"
          and a.lower()!= 'trabajo' and a.lower()!= "práctico") and a.lower()!= "experiencia" and a.lower()!="alumno" and a.lower()!="Experiencia”" and a.lower()!="apellido": #heuristica 100, perdón Hernán
            flag=1
            nombre = nombre + " " + a
        elif(flag==1):
            break
    print("el nombre del alumno es:",nombre)
    textoEnOraciones = tokenize.sent_tokenize(textoPiola)
    textoEnOraciones = [item for item in textoEnOraciones if len(item)>=30] #solo frases largas
    textoEnOraciones = [item for item in textoEnOraciones if (item[0:4]!="http")] #le saco los links
    print("uno")
    listaDeTextosEnOraciones = textosEnOraciones()
    print("dos")
    contadorOracionesPlagiadas =0
    for oracion in textoEnOraciones:
        contadorOracionesPlagiadas += flagPlagio
        flagPlagio=0
        for index,texto in enumerate(listaDeTextosEnOraciones):
            for indexItem,item in enumerate(texto):
                    if(funcionCoincidencia2(oracion,item) > confianza):
                        print("oracion analizada:",oracion)
                        print("oracion que plagea:",item)
                        print("En el archivo",listaDocx[index][23:]," Oracion número(posicion relativa):",indexItem)
                        flagPlagio=1
        print("lista una oracion")
    print("el porcentaje de plagio es de:",contador/len(textoEnOraciones),"con una confianza de:",confianza *100)


In [ ]:
verFrasesPlagiadas("dataset-nlp-plagio-utn\\TP7_Dominio_de_la_informacion_-_Juan_Facundo_Obregon.docx",0.8)

el nombre del alumno es:  Facundo Obregon
uno
dos
oracion analizada: Trabajo Practico 7 – El dominio de la información.
oracion que plagea: Trabajo Practico 7 – El dominio de la información.
En el archivo TP7_Dominio_de_la_informacion_-_Juan_Facundo_Obregon.docx  Oracion número(posicion relativa): 0
oracion analizada: Trabajo Practico 7 – El dominio de la información.
oracion que plagea: Trabajo Práctico N° 4: ​“El dominio de la información”.
En el archivo UTNMktg2016-Maschwitz-TP7.docx  Oracion número(posicion relativa): 2
lista una oracion
oracion analizada: TEST DE LECTURA “EL DOMINIO DE LA INFORMACIÓN” - CAPÍTULO 2. .
oracion que plagea: TEST DE LECTURA “EL DOMINIO DE LA INFORMACIÓN” - CAPÍTULO 2. .
En el archivo TP7_Dominio_de_la_informacion_-_Juan_Facundo_Obregon.docx  Oracion número(posicion relativa): 1
oracion analizada: TEST DE LECTURA “EL DOMINIO DE LA INFORMACIÓN” - CAPÍTULO 2. .
oracion que plagea: TEST DE LECTURA “EL DOMINIO DE LA INFORMACIÓN” - CAPÍTULO 7 (Excluye los ca